In [1]:
import numpy as np
import rasterio
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [2]:
building_mask_path = r"..\data\raw\Building_Mask_Example_1.tif"
flat_roof_mask_path = r"..\data\raw\Reproj_30cm_London_Flat_Roofs_Example_1.tif"
sat_image_path = r"..\data\raw\102220103020-1030050093F91800-visual.tif"

In [3]:
building_mask = cv2.imread(building_mask_path, cv2.IMREAD_UNCHANGED)
building_mask = building_mask.astype(np.uint8)

flat_roof_mask = cv2.imread(flat_roof_mask_path, cv2.IMREAD_UNCHANGED)
flat_roof_mask = flat_roof_mask.astype(np.uint8)

sat_image = cv2.imread(sat_image_path, cv2.COLOR_BGR2RGB)
sat_image = sat_image.astype(np.uint8)

In [4]:
building_contours, contour_hierarchy = cv2.findContours(building_mask, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

In [73]:
# Initialise a black image to be filled with contours
out = np.zeros((building_mask.shape[0], building_mask.shape[1]))

# Iterate through the contours, adding them one by one into the black image
for i, contour in enumerate(building_contours):
    if (contour_hierarchy[0, i, 3] == -1):
        cv2.drawContours(out, [contour], -1, 1, 2)

In [12]:
# Adapted from https://www.geeksforgeeks.org/python-opencv-find-center-of-contour/
for i, contour in enumerate(building_contours):
    if (contour_hierarchy[0, i, 3] == -1):

        M = cv2.moments(contour)

        if M['m00'] != 0:

            centre_x = int(M['m10']/M['m00'])
            centre_y = int(M['m01']/M['m00'])

            if (256 < centre_x < building_mask.shape[0] - 256) and (256 < centre_y < building_mask.shape[1] - 256):

                sat_image_mask = np.zeros((sat_image.shape[0], sat_image.shape[1], 3)).astype(np.uint8)
                cv2.drawContours(sat_image_mask, [contour], -1, color=(255, 255, 255), thickness=cv2.FILLED)
                masked_sat_image = cv2.bitwise_and(sat_image, sat_image_mask)

                left_boundary = centre_x - 256
                right_boundary = centre_x + 256            
                bottom_boundary = centre_y - 256
                top_boundary = centre_y + 256
                
                sat_img_tile = masked_sat_image[left_boundary:right_boundary, bottom_boundary:top_boundary, :]
                cv2.imwrite(f"../data/interim/sat_image_mask_{i}.tif", sat_image_mask)
                if i > 1000:
                    break
            

KeyboardInterrupt: 

In [8]:
cv2.imwrite(r"..\data\interim\sat_image_tile.png", sat_img_tile)

True

In [9]:
cv2.imwrite(r"..\data\interim\sat_image_mask.tif", sat_image_mask)

True